# Jurados Electorales de  PDF a CSV 
> Un tutorial de como comvertir PDF a CSV

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

# Parser de PDF a CSV de los jurados electorales.

El objetivo de este notebook es obtener un CSV que pueda luego ser analizado y explorado.
Este notebook muestra los pasos necesarios para parsear el archivo `jurados.pdf` a CSV.

Se tubo un problema a la hora de extraer con `regular expressions` los Doc. de Identidad con valores
alfanumericos. Se deja como trabajo restanto, limpiar la columna respectiva y adyasentes a estos valores.

## Setup

Se siguen los ejemplos de https://nbviewer.jupyter.org/github/chezou/tabula-py/blob/master/examples/tabula_example.ipynb

In [1]:
# Instalar tabula-py
%pip install tabula-py

     |████████████████████████████████| 11.7 MB 1.1 MB/s eta 0:00:01    |██████████████████▊             | 6.8 MB 1.9 MB/s eta 0:00:03     |███████████████████             | 6.9 MB 1.9 MB/s eta 0:00:03     |███████████████████▎            | 7.0 MB 1.9 MB/s eta 0:00:03
Note: you may need to restart the kernel to use updated packages.


## Some imports

In [ ]:
from tqdm import tqdm
import pandas as pd
from tabula import read_pdf


## Read PDF file 

Se crea una lista de dataframes que contienen la informacion de cada pagina del pdf.

In [1]:

# Read the pdf as a list of dataframes
dfs = read_pdf("jurados.pdf", pages="all", guess=False)

print(len(dfs))
dfs[0].head(n=10)

3072


,Unnamed: 0,Unnamed: 1,ESTADO PLURINACIONAL DE BOLIVIA,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,ÓRGANO ELECTORAL PLURINACIONAL,NaN,NaN,NaN,NaN
1,NaN,NaN,Tribunal Electoral Departamental de La Paz,NaN,NaN,NaN,NaN
2,NaN,NaN,Elecciones Generales 2020,NaN,NaN,NaN,NaN
3,NaN,NaN,18 de Octubre de 2020,NaN,NaN,NaN,NaN
4,País:,Bolivia,LISTADO DE JURADOS ELECTORALES,NaN,NaN,NaN,NaN
5,N°,Apellidos y Nombres,Doc. de Identidad Municipio,Recinto,NaN,Mesa,NaN
6,1,ABALO LUQUE JUDITH NANCY,I 9102281 El Alto,Col. Rotary Chuquiago Marca,NaN,NaN,1.0
7,2,ABALOS CHOQUE MANCY,I 4960747 El Alto,Unidad Educativa Juan Capriles,NaN,NaN,1.0
8,3,ABALOS QUISPE MARUJA MERCEDES,I 4943746 El Alto,Col. Tunari,NaN,NaN,1.0
9,4,ABARIOJO YUCO MARCO ANTONIO,I 1939274 Nuestra Señora de La Paz,Col. Cristo Rey,NaN,NaN,1.0


## Processing

Se crea una copia de la lista de dataframes para no volver a cargar el archivo pdf.

In [2]:
# Create Copy
dfs_dev = dfs.copy()

Se procedio a probar diferentes paginas (su valor en la lista es su valor como pagina en el pdf). En este caso el 3041

In [390]:
# Read the page 3041  and cut the first 5 rows
df = dfs_dev[3041].iloc[5:]

# Rename columns
df.columns = df.iloc[0]

# Use as dataframe the ramaining data.
df = df[1:]

df.tail()

5,N°,Apellidos y Nombres,Doc. de Identidad Municipio,Recinto Mesa,NaN
20,54.753,ZAVALA ESPINOZA CLAUDIA ANTONIETA,I 4790366 Nuestra Señora de La Paz,Escuela Rosemari de Barrientos,46.0
21,54.754,ZAVALA HUMEREZ FEDERICO ERNESTO,I 4311981 Nuestra Señora de La Paz,Esc. San Martin,34.0
22,54.755,ZAVALA JIMENEZ MARIO ALBERTO,I 8312089 El Alto,Colegio Walter Alpire 2do Patio,14.0
23,54.756,ZAVALA JIMENEZ PAOLA ANDREA,I 10920426 El Alto,Colegio Walter Alpire 1er Patio,28.0
24,Fecha:,18/09/2020,NaN,Página: 3.042 de 3.072,NaN


Se puede ver que hay valores de "fecha -- - - - - " al final del dataframe. Posteriormente se limpiara estos valores.

In [ ]:
# En algunos dataframes, el nombre de Recinto y Mesa estan juntos, si es el caso, proceder a renombrarlo


In [391]:
# Rename column value

if "Recinto Mesa" in list(df.columns):
    df.rename(columns={"Recinto Mesa": "Recinto" }, inplace=True)


In [392]:
# Check Head

df.head()

5,N°,Apellidos y Nombres,Doc. de Identidad Municipio,Recinto,NaN
6,54.739,ZARSURI LUNA LIMBER,I 7015364 Cairoma,U.E. Araca Torre Pampa,3.0
7,54.740,ZARSURI RIASA JULIA,I 6805130 Inquisivi,Escuela Eliodoro Camacho,5.0
8,54.741,ZARSURI SALAZAR CINTHYA STEPHANIE,I 6894259 Nuestra Señora de La Paz,Escuela Superior de Formación de Maestros Simo...,15.0
9,54.742,ZARSURI TARQUI GABRIEL FRANZ,I 4376305 Ixiamas,Esc. German Busch,15.0
10,54.743,ZARSURI TINTAYA EDITH,I 9090775 El Alto,U.E. Iberdrola,1.0


Se procede a eliminar los N ultimos elementos de la tabla, el ultimo elemento que corresponde a la fecha se lo señala con un 1.

In [343]:
# Drop n last row elements in the end of the table
df.drop(df.tail(1).index,inplace=True) 

In [344]:
# Check Tail
df.tail()

5,N°,Apellidos y Nombres,Doc. de Identidad Municipio,Recinto Mesa,NaN
19,54.374,ZAMBRANA FLORES BORIS PABLO,I 2622984 Nuestra Señora de La Paz,Unidad Educativa Los Pinos,26.0
20,54.375,ZAMBRANA FLORES LUIS MIGUEL,I 6827045 Nuestra Señora de La Paz,Esc. Jose Santos,21.0
21,54.376,ZAMBRANA FUENTES JOEL OMAR,I 9947828 Viacha,Esc. San Luis,22.0
22,54.377,ZAMBRANA GALARZA QUISPE PRIMITIVA,I 6130700 El Alto,Colegio 6 de Junio,26.0
23,54.378,ZAMBRANA GARCIA OSVALDO ENRIQUE,I 5501507 Nuestra Señora de La Paz,Escuela Pedro Poveda,17.0


### REGEX 

Se utiliza regular expresion para separar los valores de la columna "Doc. de Identidad Municipio".

Es aqui donde se tubo problmeas para parsear los valores alfanumericos que podria tener un Documento de Identidad.

In [345]:
# Test Regex
df["Doc. de Identidad Municipio"].str.split(r"\b(\d+)\b([^\w-])", expand=True)


,0,1,2,3
6,I,6824163,,Achocalla
7,I,8461434,,Nuestra Señora de La Paz
8,I,5497883,,Nuestra Señora de La Paz
9,I,6936357,,Apolo
10,I,7313138,,El Alto
11,I,6794431,,Nuestra Señora de La Paz
12,I,1883348,,El Alto
13,I,4891939,,Nuestra Señora de La Paz
14,I,13927008,,El Alto
15,I,12670368,,El Alto


Se crean 4 columnas adicionales para alojar los valores que el REGEX encontro

In [346]:

df[["PREFIX - Doc. de Identidad", "Doc. de Identidad", "unnamed:0", "Municipio"]] = df["Doc. de Identidad Municipio"].str.split(r"\b(\d+)\b([^\w-])", expand=True)

Revisar cuantos valores `nan` se tienen en los nombres de las columnas

In [347]:
print(len(df.columns))
for c in df.columns:
    print(type(c), c)

9
<class 'str'> N°
<class 'str'> Apellidos y Nombres
<class 'str'> Doc. de Identidad Municipio
<class 'str'> Recinto Mesa
<class 'numpy.float64'> nan
<class 'str'> PREFIX - Doc. de Identidad
<class 'str'> Doc. de Identidad
<class 'str'> unnamed:0
<class 'str'> Municipio


### Limpiar los valores nan de los nombres de las columnas

Se renombrar los valores de los nombres de las columnas que tubieran valores `nan` con placeholders denominados `unnamed:X` donde X es un indice que se autoincrementa por el numero de valores `nan`  presentes en las columnas.

In [349]:
# Fill the nan values in column names

# List hte actual column names
df_names_to_fix = pd.Series(df.columns)

# Create a new list with fixed column names
df_names_fixed = df_names_to_fix.fillna('unnamed:' + (df_names_to_fix.groupby(df_names_to_fix.isnull()).cumcount() + 1).astype(str))

# Set the new column names to the test dataframe
df.columns  = df_names_fixed


In [350]:
# Check head
df.head()

5,N°,Apellidos y Nombres,Doc. de Identidad Municipio,Recinto Mesa,unnamed:1,PREFIX - Doc. de Identidad,Doc. de Identidad,unnamed:0,Municipio
6,54.361,ZAMBRANA CHEJO INES AMPARO,I 6824163 Achocalla,U. E. Marquirivi,14.0,I,6824163,,Achocalla
7,54.362,ZAMBRANA CHOQUE JOSE ALFREDO,I 8461434 Nuestra Señora de La Paz,Escuela Superior de Formación de Maestros Simo...,15.0,I,8461434,,Nuestra Señora de La Paz
8,54.363,ZAMBRANA CLAUDIA PAMELA,I 5497883 Nuestra Señora de La Paz,Esc. Sagrado Corazon De Jesus,35.0,I,5497883,,Nuestra Señora de La Paz
9,54.364,ZAMBRANA COLQUE JHANETH CATALINA,I 6936357 Apolo,U. E. Machua,1.0,I,6936357,,Apolo
10,54.365,ZAMBRANA COLQUE VANIA,I 7313138 El Alto,Col. Santa Maria De Los Angeles,35.0,I,7313138,,El Alto


In [414]:
# Check new column names

print(len(df.columns))
for c in df.columns:
    print(type(c), c)

5
<class 'str'> N°
<class 'str'> Apellidos y Nombres
<class 'str'> Doc. de Identidad Municipio
<class 'str'> Recinto
<class 'numpy.float64'> nan


In [352]:
df["unnamed:0"].unique()

array([' '], dtype=object)

### Reemplazar los valores de Mesa por los extraidos

Como ya se tiene extraido el valor de mesa en una columna auxiliar , en este caso `"unnamed:1"`, se procede a coloar su valor en su columna respectiva.

In [259]:
df["Mesa"] = df["unnamed:1"]

Se quitan las columnas auxiliares.

In [353]:
df.drop(columns=["Doc. de Identidad Municipio",  "unnamed:0"], inplace=True)

In [354]:
# Check head

df.head()

5,N°,Apellidos y Nombres,Recinto Mesa,unnamed:1,PREFIX - Doc. de Identidad,Doc. de Identidad,Municipio
6,54.361,ZAMBRANA CHEJO INES AMPARO,U. E. Marquirivi,14.0,I,6824163,Achocalla
7,54.362,ZAMBRANA CHOQUE JOSE ALFREDO,Escuela Superior de Formación de Maestros Simo...,15.0,I,8461434,Nuestra Señora de La Paz
8,54.363,ZAMBRANA CLAUDIA PAMELA,Esc. Sagrado Corazon De Jesus,35.0,I,5497883,Nuestra Señora de La Paz
9,54.364,ZAMBRANA COLQUE JHANETH CATALINA,U. E. Machua,1.0,I,6936357,Apolo
10,54.365,ZAMBRANA COLQUE VANIA,Col. Santa Maria De Los Angeles,35.0,I,7313138,El Alto


## Crear el pipeline para todos las páginas.

Con las herramientas creadas, se procede a crear un pipeline para extrar todos todas las tablas de todas las paginas.

### Crear funciones auxiliares 

In [404]:
def fix_nan_column_names(df_column_names):
    """
    Create a clean column names, where nan are replace by unnamed:X value, where
    X is an index for each nan value found.
    """
    # List hte actual column names
    df_names_to_fix = pd.Series(df_column_names)

    # Create a new list with fixed column names
    df_names_fixed = df_names_to_fix.fillna('unnamed:' + (df_names_to_fix.groupby(df_names_to_fix.isnull()).cumcount() + 1).astype(str))

    return df_names_fixed

def clean_dataframe(df_input):
    """
    Function para limpiar el dataframe que proviene de read_pdf .
    
    Se procesa los nombres de las columnas segun las variaciones de tamaño y nombres que pueda tener
    el dataframe input
    
    PARAMS:
    -------
    df_input: Dataframe que proviene del parser read_pdf
    
    RETURNS:
    -------
    df :    Dataframe que fue procesador y limpiado.
    
    """
    
    # Create a copy to work on of the dataframe
    df = df_input.copy()
    
    # Remove datetime row
    df.drop(df.tail(1).index,inplace=True) # drop last n rows

    # Delete the first 5 rows
    df = df.iloc[5:]

    # Set the column name to the first element of this new rows
    df.columns = df.iloc[0]

    # Start the rows from the next one element 
    # since we choose 0 as the new column names
    df = df[1:]
    
    # If recinto mesa is in column names, change his name.
    if "Recinto Mesa" in list(df.columns):
        df.rename(columns={"Recinto Mesa": "Recinto" }, inplace=True)
    
    # REGEX part
    # Create New Columns spliting the nested one "Doc. de Identidad Municipio" using regex
    df[["PREFIX - Doc. de Identidad", "Doc. de Identidad", "unnamed:0", "Municipio"]] = df["Doc. de Identidad Municipio"].str.split(r"\b(\d+)\b([^\w-])", expand=True)
    
    # List The actual column names
    new_colum_names = fix_nan_column_names(df.columns)

    # First kind of Variation for column names
    if (len(new_colum_names) == 11) or (len(new_colum_names) == 10):

        # Set the new column names to the test dataframe
        df.columns  = new_colum_names
        try:
            # Set the value from this "Mesa_Aux" to the "Mesa" column
            df["Mesa"] = df["unnamed:2"]
            df.drop(columns=["Doc. de Identidad Municipio", "unnamed:2", "unnamed:0", "unnamed:1"], inplace=True)
        
        except Exception as e:
            
            # Set the value from this "Mesa_Aux" to the "Mesa" column
            df["Mesa"] = df["unnamed:1"]
            df.drop(columns=["Doc. de Identidad Municipio", "unnamed:0", "unnamed:1"], inplace=True)
            
        finally:
            return df
        
    # Second Kind of variation for column names
    if len ( new_colum_names) == 12:

        # Set the new column names to the test dataframe
        df.columns  = new_colum_names
                
        # Drop Unused column names
        df.drop(columns=["Doc. de Identidad Municipio",  "unnamed:3", "unnamed:2", "unnamed:0", "unnamed:1"], inplace=True)
        return df  
    
    # Third Kind of Variation for column names
    if len ( new_colum_names) == 9:
        df.columns  = new_colum_names
        try:
            df["Mesa"] = df["unnamed:1"]
            df.drop(columns=["Doc. de Identidad Municipio", "unnamed:1", "unnamed:0"], inplace=True)
        except:
            df.drop(columns=["Doc. de Identidad Municipio", "unnamed:0"], inplace=True)
        finally:
            return df   

    
    # If no match for new_colum_names
    # print len for debug later
    print(len ( new_colum_names))
    return None

In [405]:
# Probar la funcion con el valor 3041  "pagina 3041" en el PDF

df_clean = clean_dataframe(dfs[3041])
df_clean.tail()

5,N°,Apellidos y Nombres,Recinto,PREFIX - Doc. de Identidad,Doc. de Identidad,Municipio,Mesa
19,54.752,ZARZURI TENORIO DIEGO HERLAND,Colegio Mariscal Santa Cruz,I,9173050,Achacachi,21.0
20,54.753,ZAVALA ESPINOZA CLAUDIA ANTONIETA,Escuela Rosemari de Barrientos,I,4790366,Nuestra Señora de La Paz,46.0
21,54.754,ZAVALA HUMEREZ FEDERICO ERNESTO,Esc. San Martin,I,4311981,Nuestra Señora de La Paz,34.0
22,54.755,ZAVALA JIMENEZ MARIO ALBERTO,Colegio Walter Alpire 2do Patio,I,8312089,El Alto,14.0
23,54.756,ZAVALA JIMENEZ PAOLA ANDREA,Colegio Walter Alpire 1er Patio,I,10920426,El Alto,28.0


### Crear funcion para el pipeline

In [372]:
def create_clean_dfs(dfs):
    """
        Funcion para procesar y limpiear una lista de dataframes.
        PARAMS:
        ------
        dfs: List of Dataframes
        
        RETURNS:
        -------
        status : dict , donde cada uno de los KEYS, corresponde los nombres "FAIL" u "GOOD"
                        y los VALUES una lista de dataframes que fueron procesados
                        correctamente y los que fallaron.
    """

    status = {
        "TO_FIX_DATAFRAME": [],
        "DFS_CLEAN":[]
    }
    
    # Iterate over the list of dataframes
    for df_raw in tqdm(dfs):
        
        # Use the function for clean the dataframe
        df_clean = clean_dataframe(df_raw)
        
        # if dataframe is NOne
        if df_clean is None:
            # Appen this to the list of fails
            status["TO_FIX_DATAFRAME"].append(df_clean)
            
            # Break the loop for debug what just happened
            break
        else:
            # Procede to append the clean dataframe to the 
            #  DFS_CLEAN dataframe list
            
            status["DFS_CLEAN"].append(df_clean)

    print(f"""
        FAIL: {len(status["TO_FIX_DATAFRAME"])},
        GOOD: {len(status["DFS_CLEAN"])}
    """)
    
    return status

In [407]:
status = create_clean_dfs(dfs_dev)


100%|██████████| 3072/3072 [00:30<00:00, 99.70it/s] 


        FAIL: 0,
        GOOD: 3072
    


### Concatenate Dataframe

Ya que se tiene una lista de dataframes uniformes, se los pasara a concatenar en un solo dataframe.

In [408]:
# Concat over Columns axis the list of  dataframes

dfs_clean = pd.concat(status["DFS_CLEAN"])

# Check Shape
print(dfs_clean.shape)

# Check Head
dfs_clean.head()

(55284, 7)


,N°,Apellidos y Nombres,Recinto,Mesa,PREFIX - Doc. de Identidad,Doc. de Identidad,Municipio
6,1,ABALO LUQUE JUDITH NANCY,Col. Rotary Chuquiago Marca,1,I,9102281,El Alto
7,2,ABALOS CHOQUE MANCY,Unidad Educativa Juan Capriles,1,I,4960747,El Alto
8,3,ABALOS QUISPE MARUJA MERCEDES,Col. Tunari,1,I,4943746,El Alto
9,4,ABARIOJO YUCO MARCO ANTONIO,Col. Cristo Rey,1,I,1939274,Nuestra Señora de La Paz
10,5,ABASTO ARANIBAR MAURICIO WILSON,Colegio Dora Schmidt,1,I,9196248,Nuestra Señora de La Paz


In [409]:
# Check Tail
dfs_clean.tail()

,N°,Apellidos y Nombres,Recinto,Mesa,PREFIX - Doc. de Identidad,Doc. de Identidad,Municipio
7,55.280,ZURITA VALLEJOS GROBER,U. E. 16 de Julio de Mapiri,15,I,8824037,Mapiri
8,55.281,ZURITA VILLCA JOB MARCELO,Escuela Puerto Perez,6,I,9239461,Puerto Pérez
9,55.282,ZURITA ZABALETA JEANNETH MARY,Colegio Don Bosco,44,I,4289623,El Alto
10,55.283,ZURITA ZELADA ADALID,Liceo Bolivia,7,I,3768774,Nuestra Señora de La Paz
11,55.284,ZUZAÑO FLORES MARCELO,Esc. Mscal. Antonio Jose De Sucre,4,I,10064472,Nuestra Señora de La Paz


### Save to CSV

In [412]:
# Save the dataframe as CSV
dfs_clean.to_csv("JURADOS_CLEAN.csv", index=False)